# DQM Maximization of COVID Vaccine Distribution
We applied a DQM solution of the Knapsack problem to the issue of COVID-19 vaccine distribution. Specifically, we focused on how vaccines arriving in the United States would be distributed to individual states. 

In this model, the federal vaccine distribution center is receiving offers from states to ship a given number of vaccines. The distribution center's goal is to maximize the number of people who receive the vaccine. It must consider the rate of vaccine administration per state as well as the sizes of the shipment offers to put as many vaccines as possible to good use.

Let' start by importing the tools we need: a shipment mocker to generate some shipment offers, an shipment analyzer to estimate how many administrations will occur, and a knapsack solver to find the optimal solution.

In [ ]:
from shipment_mocking import create_mock_shipment
from shipment_analysis import estimate_shipment_value
from knapsack_solver import KnapsackSolver

Next, let's set up some constants for this scenario. Since we aren't in charge of real shipments, we need to generate a number of fake shipment offers and imagine we have a supply of vaccines with which to fulfill them. Since we're dealing with large numbers here, the scaling factor helps make the numbers a bit nicer for computation.

In [ ]:
NUM_SHIPMENTS = 10
NUM_VACCINE_DOSES = 200_000
SCALING_FACTOR = 10_000.

It's time to create the shipment offers. These are the items of the knapsack problem. The weight is the number of doses to be shipped and the value is the number of doses expected to be administered.

In [ ]:
shipments = []

for _ in range(NUM_SHIPMENTS):
    shipment = create_mock_shipment()
    weight = shipment["doses"]
    value = estimate_shipment_value(shipment)

    shipments.append((shipment, weight, value))
    print(f"Shipment: {shipment}, value: {value}")

Now we have everything we need to set up the knapsack problem. Our solver makes it easy; simply create a new solver and add each item with its weight and value.

In [ ]:
knapsack = KnapsackSolver()

for shipment, weight, value in shipments:
    knapsack.add_item(shipment, weight / SCALING_FACTOR, value / SCALING_FACTOR)

To run the DQM solver, you need to configure access to D-Wave's online solvers.  You can create a configuration file using the interactive CLI as described [here](https://docs.ocean.dwavesys.com/en/stable/overview/sapi.html#creating-a-configuration-file).

If you want to see the details of the DQM before submitting it to the solver, you can change the last argument to `debug=True`. 

In [ ]:
print("Solving... ", end="")
item_list, total_value, total_weight = knapsack.solve(NUM_VACCINE_DOSES / SCALING_FACTOR, debug=False)
print("Done.")

Let's take a look at the final solution:

In [ ]:
total_doses = 0
for item, quantity in item_list:
    print(f"{quantity} shipment{'s' if quantity != 1 else ''} of {item['doses']} doses delivered to {item['state']}")
    total_doses += item["doses"]

print(f"\nOut of {NUM_VACCINE_DOSES} doses available, {int(total_weight * SCALING_FACTOR)} delivered, {total_value * SCALING_FACTOR} administered")

Interested in seeing other potential outcomes? Click the Run All Cells button to explore another randomly generated scenario.